In [1]:
import os

In [2]:
import sys

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from scipy.io import loadmat

In [7]:
#import pyod packages and methods

In [9]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [10]:
#import Metrics Package

In [11]:
from pyod.utils import standardizer
from pyod.utils import precision_n_scores
from sklearn.metrics import roc_auc_score

In [23]:
# Define data file and read X & Y

In [13]:
mat_file_list = ["arrhythmia.mat","cardio.mat","glass.mat",
                 "ionosphere.mat","letter.mat","mnist.mat",
                 "musk.mat","optdigits.mat","pendigits.mat",
                 "pima.mat","satellite.mat","satimage-2.mat",
                 "shuttle.mat","vertebral.mat","vowels.mat",
                 "wbc.mat"]

In [38]:
# Loading the mat file

In [15]:
data = loadmat("D:\LetsUpgrade\Project1(AnomalyDetection)\cardio.mat")

In [16]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [17]:
len(data)

5

In [18]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [20]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

In [33]:
# Input(Independant) feature shape in mat file format

In [34]:
type(data["X"])

numpy.ndarray

In [24]:
data["X"].shape

(1831, 21)

In [35]:
# Output(Dependant) feature shape in mat file format

In [36]:
type(data["y"])

numpy.ndarray

In [37]:
data["y"].shape

(1831, 1)

In [40]:
df_columns=['Data','#Sample','#Dimensions','Outlier Perc',
            'PCA','MCD','OCSVM','LOF','CBLOF','KNN','HBOS',
            'ABOD','IFOREST','FEATUREBAGGING']

In [41]:
df_columns

['Data',
 '#Sample',
 '#Dimensions',
 'Outlier Perc',
 'PCA',
 'MCD',
 'OCSVM',
 'LOF',
 'CBLOF',
 'KNN',
 'HBOS',
 'ABOD',
 'IFOREST',
 'FEATUREBAGGING']

In [48]:
# ROC(Region of Characteristics) Performance evolution table

In [49]:
roc_df=pd.DataFrame(columns=df_columns)
roc_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING


In [52]:
# Precision_n_scoresPerformance evolution table

In [44]:
prn_df=pd.DataFrame(columns=df_columns)
prn_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING


In [53]:
# Time Performace evolution table

In [54]:
time_df=pd.DataFrame(columns=df_columns)
time_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING


In [55]:
# Exploring all mat files

In [64]:
from time import time
random_state = np.random.RandomState(42)
for mat_file in mat_file_list:
    print("\n... Processing",mat_file,'.....')
    mat = loadmat(os.path.join("D:\LetsUpgrade\Project1(AnomalyDetection)",mat_file))
    
    X = mat['X']
    y = mat['y'].ravel()
    
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = random_state)
    
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
    
    classifiers = {'Angle-based Outlier Detector (ABOD)' : ABOD(contamination=outliers_fraction), 
                  'Cluster-based Local Outlier Factor': CBLOF(contamination=outliers_fraction,check_estimator=False ,random_state=random_state),
                  'Feature Bagging': FeatureBagging(contamination=outliers_fraction, random_state=random_state),
                  'Historgram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
                  'Isolation Forest' : IForest(contamination=outliers_fraction, random_state=random_state),
                  'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
                  'Local Outlier Factor (LOF)' : LOF(contamination=outliers_fraction),
                  'Minimum Covariance Determinant (MCD)': MCD(contamination=outliers_fraction, random_state=random_state),
                  'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
                  'Principal Component Analysis (PCA)': PCA(contamination=outliers_fraction, random_state=random_state)}
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1-t0, 4)
        time_list.append(duration)
        
        roc = round(roc_auc_score(y_test, test_scores), 4)
        prn = round(precision_n_scores(y_test, test_scores), 4)
        print(f"{clf_name} ROC: {roc}, precision @ rank n: {prn}, execution time: {duration}s' ")
        
        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat .....
Angle-based Outlier Detector (ABOD) ROC: 0.7687, precision @ rank n: 0.3571, execution time: 5.3464s' 
Cluster-based Local Outlier Factor ROC: 0.7684, precision @ rank n: 0.4643, execution time: 3.594s' 
Feature Bagging ROC: 0.7799, precision @ rank n: 0.5, execution time: 0.8996s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.8511, precision @ rank n: 0.5714, execution time: 3.5801s' 
Isolation Forest ROC: 0.8527, precision @ rank n: 0.5714, execution time: 0.7939s' 
K Nearest Neighbors (KNN) ROC: 0.782, precision @ rank n: 0.5, execution time: 0.1277s' 
Local Outlier Factor (LOF) ROC: 0.7787, precision @ rank n: 0.4643, execution time: 0.1326s' 


C:\Users\admin\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC: 0.8228, precision @ rank n: 0.4286, execution time: 2.1891s' 
One-class SVM (OCSVM) ROC: 0.7986, precision @ rank n: 0.5, execution time: 0.0878s' 
Principal Component Analysis (PCA) ROC: 0.7997, precision @ rank n: 0.5, execution time: 0.1157s' 

... Processing cardio.mat .....
Angle-based Outlier Detector (ABOD) ROC: 0.5763, precision @ rank n: 0.1875, execution time: 0.5555s' 
Cluster-based Local Outlier Factor ROC: 0.8221, precision @ rank n: 0.4844, execution time: 0.2953s' 
Feature Bagging ROC: 0.4879, precision @ rank n: 0.1406, execution time: 1.118s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.8453, precision @ rank n: 0.4688, execution time: 0.009s' 
Isolation Forest ROC: 0.9414, precision @ rank n: 0.5, execution time: 0.5277s' 
K Nearest Neighbors (KNN) ROC: 0.6959, precision @ rank n: 0.2812, execution time: 0.253s' 
Local Outlier Factor (LOF) ROC: 0.4715, precision @ rank n: 0.125, execution time: 0.1307s' 


C:\Users\admin\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Users\admin\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-152.076751858648862 > -250.756822706501453). You may want to try with a higher value of support_fraction (current value: 0.511).
  RuntimeWarning)


Minimum Covariance Determinant (MCD) ROC: 0.8778, precision @ rank n: 0.3906, execution time: 0.7243s' 
One-class SVM (OCSVM) ROC: 0.9507, precision @ rank n: 0.5938, execution time: 0.1047s' 
Principal Component Analysis (PCA) ROC: 0.9638, precision @ rank n: 0.6875, execution time: 0.0509s' 

... Processing glass.mat .....
Angle-based Outlier Detector (ABOD) ROC: 0.7104, precision @ rank n: 0.25, execution time: 0.0638s' 
Cluster-based Local Outlier Factor ROC: 0.8506, precision @ rank n: 0.25, execution time: 0.0628s' 
Feature Bagging ROC: 0.7043, precision @ rank n: 0.25, execution time: 0.0499s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.6524, precision @ rank n: 0.0, execution time: 0.0039s' 
Isolation Forest ROC: 0.7195, precision @ rank n: 0.25, execution time: 0.3989s' 
K Nearest Neighbors (KNN) ROC: 0.7805, precision @ rank n: 0.25, execution time: 0.012s' 
Local Outlier Factor (LOF) ROC: 0.7774, precision @ rank n: 0.25, execution time: 0.004s' 
Minimum Covariance Dete

...

Minimum Covariance Determinant (MCD) ROC: 0.3813, precision @ rank n: 0.0, execution time: 1.7513s' 
One-class SVM (OCSVM) ROC: 0.5089, precision @ rank n: 0.0, execution time: 1.9469s' 
Principal Component Analysis (PCA) ROC: 0.5076, precision @ rank n: 0.0, execution time: 0.0936s' 

... Processing pendigits.mat .....
Angle-based Outlier Detector (ABOD) ROC: 0.6012, precision @ rank n: 0.0, execution time: 2.1293s' 
Cluster-based Local Outlier Factor ROC: 0.9641, precision @ rank n: 0.2727, execution time: 0.369s' 
Feature Bagging ROC: 0.3915, precision @ rank n: 0.0182, execution time: 4.7543s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.9178, precision @ rank n: 0.2545, execution time: 0.012s' 
Isolation Forest ROC: 0.9478, precision @ rank n: 0.3273, execution time: 0.9853s' 
K Nearest Neighbors (KNN) ROC: 0.6964, precision @ rank n: 0.0182, execution time: 0.9495s' 
Local Outlier Factor (LOF) ROC: 0.4032, precision @ rank n: 0.0, execution time: 0.9255s' 
Minimum Covariance 

C:\Users\admin\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.615819653203957 > -75.352533689916143). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.057877525259613 > -78.640969878489642). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.222933581673360 > -77.655542768808118). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\U

Minimum Covariance Determinant (MCD) ROC: 0.9903, precision @ rank n: 0.7452, execution time: 14.3008s' 
One-class SVM (OCSVM) ROC: 0.9919, precision @ rank n: 0.9555, execution time: 64.4407s' 
Principal Component Analysis (PCA) ROC: 0.9901, precision @ rank n: 0.9499, execution time: 0.0448s' 

... Processing vertebral.mat .....
Angle-based Outlier Detector (ABOD) ROC: 0.6691, precision @ rank n: 0.1538, execution time: 0.0658s' 
Cluster-based Local Outlier Factor ROC: 0.5412, precision @ rank n: 0.1538, execution time: 0.0868s' 
Feature Bagging ROC: 0.5968, precision @ rank n: 0.2308, execution time: 0.0469s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.3146, precision @ rank n: 0.0769, execution time: 0.0029s' 
Isolation Forest ROC: 0.4717, precision @ rank n: 0.1538, execution time: 0.3879s' 
K Nearest Neighbors (KNN) ROC: 0.5728, precision @ rank n: 0.1538, execution time: 0.018s' 
Local Outlier Factor (LOF) ROC: 0.6043, precision @ rank n: 0.1538, execution time: 0.005s' 
Mi

C:\Users\admin\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-23.697386830016111 > -41.479931533394790). You may want to try with a higher value of support_fraction (current value: 0.528).
  RuntimeWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-23.704715026822193 > -41.285094902683184). You may want to try with a higher value of support_fraction (current value: 0.528).
  RuntimeWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-23.757620393124419 > -42.742548026556058). You may want to try with a higher value of support_fraction (current value: 0.528).
  RuntimeWarning)


Angle-based Outlier Detector (ABOD) ROC: 0.9498, precision @ rank n: 0.3125, execution time: 0.3939s' 
Cluster-based Local Outlier Factor ROC: 0.8038, precision @ rank n: 0.0625, execution time: 0.1277s' 
Feature Bagging ROC: 0.8721, precision @ rank n: 0.125, execution time: 0.371s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.6143, precision @ rank n: 0.0, execution time: 0.007s' 
Isolation Forest ROC: 0.6521, precision @ rank n: 0.0, execution time: 0.5196s' 
K Nearest Neighbors (KNN) ROC: 0.9573, precision @ rank n: 0.25, execution time: 0.0977s' 
Local Outlier Factor (LOF) ROC: 0.8779, precision @ rank n: 0.1875, execution time: 0.0439s' 
Minimum Covariance Determinant (MCD) ROC: 0.6865, precision @ rank n: 0.0625, execution time: 1.0133s' 
One-class SVM (OCSVM) ROC: 0.7423, precision @ rank n: 0.0625, execution time: 0.0449s' 
Principal Component Analysis (PCA) ROC: 0.5584, precision @ rank n: 0.0, execution time: 0.003s' 

... Processing wbc.mat .....
Angle-based Outlier Det

In [65]:
roc_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5778,0.845,0.8409,0.7499,0.5744,0.48
0,mnist,7603,100,9.2069,0.7741,0.8343,0.7032,0.5466,0.7936,0.8385,0.7058,0.7439,0.8355,0.8389
0,musk,3062,166,3.1679,0.1611,1,0.651,1,0.9999,0.8584,0.6679,1,1,1
0,optdigits,5216,64,2.8758,0.4902,0.6022,0.3995,0.8461,0.7395,0.3491,0.4162,0.3813,0.5089,0.5076
0,pendigits,6870,16,2.2707,0.6012,0.9641,0.3915,0.9178,0.9478,0.6964,0.4032,0.8203,0.9261,0.9384
0,pima,768,8,34.8958,0.6521,0.6192,0.5798,0.6467,0.6102,0.6581,0.586,0.6255,0.5529,0.6058


In [66]:
prn_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.5,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,mnist,7603,100,9.2069,0.3597,0.3849,0.3129,0.0899,0.2518,0.4065,0.3201,0.1942,0.3777,0.3741
0,musk,3062,166,3.1679,0.0882,1,0.5,1,0.9412,0.5,0.3235,1,1,0.9706
0,optdigits,5216,64,2.8758,0.0189,0,0.0377,0.2264,0.0377,0,0.0189,0,0,0
0,pendigits,6870,16,2.2707,0,0.2727,0.0182,0.2545,0.3273,0.0182,0,0.0182,0.2727,0.2909
0,pima,768,8,34.8958,0.4579,0.4206,0.4206,0.4673,0.4299,0.4766,0.4112,0.4673,0.4299,0.4486


In [67]:
time_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,5.3464,3.594,0.8996,3.5801,0.7939,0.1277,0.1326,2.1891,0.0878,0.1157
0,cardio,1831,21,9.6122,0.5555,0.2953,1.118,0.009,0.5277,0.253,0.1307,0.7243,0.1047,0.0509
0,glass,214,9,4.2056,0.0638,0.0628,0.0499,0.0039,0.3989,0.012,0.004,0.083,0.002,0.003
0,ionosphere,351,33,35.8974,0.0908,0.0798,0.1067,0.0289,0.4239,0.0229,0.009,0.0898,0.01,0.0389
0,letter,1600,32,6.25,0.5236,0.1516,1.0213,0.0129,0.5346,0.1815,0.1531,1.3455,0.1067,0.01
0,mnist,7603,100,9.2069,9.9723,1.7763,69.2429,0.0738,3.2872,9.5415,8.9571,3.8199,6.4029,0.2714
0,musk,3062,166,3.1679,3.0907,0.5515,16.4919,0.0788,1.7703,2.5043,2.3637,17.2628,1.6975,0.1925
0,optdigits,5216,64,2.8758,3.6433,0.6961,15.6571,0.0429,1.2836,2.4016,2.2809,1.7513,1.9469,0.0936
0,pendigits,6870,16,2.2707,2.1293,0.369,4.7543,0.012,0.9853,0.9495,0.9255,2.605,1.2197,0.0459
0,pima,768,8,34.8958,0.2494,0.0957,0.1387,0.0039,0.4319,0.0419,0.013,0.0898,0.012,0.002
